# **【AIに関する講義パートに関する基本方針】**

AIの社会実装を巡る議論を行っていく大前提として、まずは今回より2回に分けて古典的な機械学習について復習します（その後、2回に分けて深層学習について同様に復習）。今後の予定を示すならば以下のとおりです。

なお、pythonについては「自分で既にある程度学んでいること」を前提に議論を進めていきます。そのつもりでプログラミングの基礎については、厭うことなく自ら手を動かして覚えていきましょう。

また全てgithub上にcolabのコードをアップロードしておきますので、そちらをダウンロードして受講するようにして下さい。

各回の冒頭には前回の最後に課した課題について、colab上で解いたものを受講者自身で展開してもらい、プレゼンをしてもらいます。ChatGPTに「相談」してそのままコピーするのではなく、コードの細部に至るまで質問されたらば答えられるように準備をしておいてください。

**（第7講（今回））**

●古典的な機械学習の復習その1（回帰分析、k-means、ランダムフォレスト）

**（第8回）**

●古典的な機械学習の復習その2（ロジスティック回帰、サポートベクトルマシーン（SVM））

**（第9回）**

●深層学習の復習その1（畳込みニューラルネットワーク（CNN）、強化学習）

**（第10回）**

●深層学習の復習その2（再帰型ニューラルネットワーク、LSTM、自然言語処理の各種手法）

第11講においては大規模言語モデル（LLM）としてgpt-4.1等を実際に用いつつ、AIエージェントの実装に向けた議論を展開します。

# **【回帰分析（重回帰分析（Multiple Regression Analysis）】**

In [ ]:
#まずは対象となるデータをダウンロードする。
#具体的には、UC バークレー大学の UCI Machine Leaning Repository にて公開されている、「Wine Quality Data Set (ワインの品質)」の赤ワインのデータセット。

!wget http://pythondatascience.plavox.info/wp-content/uploads/2016/07/winequality-red.csv


In [ ]:
import pandas as pd

df = pd.read_csv('winequality-red.csv', sep=';')
df.head()

#データはダウンロードしたらばまずはじっくり見てみる癖をつけること。

In [22]:
x = df[['density', 'pH']]
y = df[['alcohol']]
x1 = df[['density']]
x2 = df[['pH']]

print(x.shape)
print(y.shape)

(1599, 2)
(1599, 1)


In [ ]:
#ここで必要になってくるのが「正規化」と言う作業。
#以下ではsklearn上のstandard scalerを用いてみる。

from sklearn import preprocessing

sscaler = preprocessing.StandardScaler()
sscaler.fit(x)
xss_sk = sscaler.transform(x)
sscaler.fit(y)
yss_sk = sscaler.transform(y)

print(xss_sk)
print(yss_sk)

**【課題】**

正規化には上記以外の方法もある。検索して探し、pytyon上で実際に演算を実行して示せ。

In [ ]:
#早速、上記の正規化したデータを用いて重回帰分析を施してみる。

from sklearn.linear_model import LinearRegression
import numpy as np

model_lr_std = LinearRegression()
model_lr_std.fit(xss_sk, yss_sk)

print(model_lr_std.coef_)
print(model_lr_std.intercept_)
print(model_lr_std.score(xss_sk, yss_sk))

#値はそれぞれ回帰係数、切片、決定係数を指す。
#回帰係数は、説明変数が目的変数にどれくらい影響するかを表す係数。重回帰分析では各変数同士の影響度を示す。
#回帰係数が大きいほど、説明変数が目的変数に強い影響を与えていることを示唆している。
#決定係数は、0から1の間の値を取り、1に近いほど回帰式がデータに適合していることを示す。

**【課題】**

上記で演算した結果をグラフ化することを試みよ。

# **【K-means法】**

クラスタリングとは、ある集合を何らかの規則によって分類することを指す。機械学習においてクラスタリングは、「教師なし学習」に分類される。

クラスタリングの計算方法はいくつかあるが、サンプル同士の類似性に基づいてグルーピングしている。クラスタリングの計算方法を大きく分類すると、「階層クラスタリング」と「非階層クラスタリング」の2つに分けられる。
今回実装するK-means法は「非階層クラスタリング」に分類される。

K-means法とはクラスターの平均を用いて、あらかじめ決められたクラスター数に分類手法を指す。K-means法のアルゴリズム概要は下記のとおり：

**●クラスタの中心の初期値をk個決める**

**●全てのサンプルとk個のクラスタとの中心距離を求め、最も近いクラスタに分類する**

**●形成されたk個のクラスタの中心を求める**

**●中心が変化しなくなるまで上記2番目と3番目の工程を繰り返す**

In [31]:
#まずは必要なライブラリーをダウンロードする。

# 必要なライブラリーのインストール
import numpy as np
import pandas as pd

# 可視化
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set_style('whitegrid')

# 正規化のためのクラス
from sklearn.preprocessing import StandardScaler

# k-means法に必要なものをインポート
from sklearn.cluster import KMeans

In [ ]:
#次に有名なirisのデータセットをダウンロードする。

# irisデータ
from sklearn.datasets import load_iris

# データ読み込み
iris = load_iris()
iris.keys()

# データフレームに格納
df_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
df_iris['target'] = iris.target # アヤメの種類（正解ラベル）
df_iris.head()

**【課題】**

このデータセットについて、（1）petal_length, petal_widthの二変数で可視化し、さらに（2）それを散布図行列で可視化せよ。

In [ ]:
#次にエルボー法でクラスター数を決めてみる。
#エルボー法はこちらを参照。https://zenn.dev/nakano_teppei/articles/e7164a385148b6

# Elbow Method
wcss = []

for i in range(1, 10):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 30, random_state = 0)
    kmeans.fit(df_iris.iloc[:, 2:4])
    wcss.append(kmeans.inertia_)


plt.plot(range(1, 10), wcss)
plt.title('The elbow method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
#実際にライブラリーを使ってモデリングしてみる。

# モデリング
clf = KMeans(n_clusters=3, random_state=1)
clf.fit(df_iris.iloc[:, 2:4])

# 学習データのクラスタ番号
clf.labels_

# 未知データに対してクラスタ番号を付与
# 今回は学習データに対して予測しているので、`clf.labels_` と同じ結果
y_pred = clf.predict(df_iris.iloc[:, 2:4])
y_pred

# 実際の種類とクラスタリングの結果を比較
fig, (ax1, ax2) = plt.subplots(figsize=(16, 4), ncols=2)

# 実際の種類の分布
ax1.scatter(df_iris['petal length (cm)'], df_iris['petal width (cm)'], c=df_iris.target, cmap=mpl.cm.jet)
ax1.set_xlabel('petal_length')
ax1.set_ylabel('petal_width')
ax1.set_title('Actual')
# クラスター分析で分類されたクラスタの分布
ax2.scatter(df_iris['petal length (cm)'], df_iris['petal width (cm)'], c=y_pred, cmap=mpl.cm.jet)
ax2.set_xlabel('petal_length')
ax2.set_ylabel('petal_width')
ax2.set_title('Predict')

**【課題】**

K-means法と同じ様な呼称のアルゴリズムに「k近傍法」がある。これらはどの様に違うのかを簡単に説明した上で、実際のデータセットを用いつつ、python上に実装し、演算を実行せよ。

# **【ランダムフォレスト】**

機械学習における分類や回帰に用いられるアルゴリズムであり、決定木を複数作成し、その結果を統合することで高い精度を実現する。

ブートストラップサンプリングと呼ばれる手法により、データセットからランダムにサンプルを取り出し、それぞれのサンプルで決定木を構築する。

最後に、各決定木の予測結果を多数決（分類）または平均（回帰）を取ることで、最終的な予測を行う。

In [37]:
#まずは必要なライブラリーをダウンロードする。

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#今回はオリジナルなデータをデータフレームに格納し、処理していく。
#従業員の特徴（年齢、職務経験、部署）に基づいて離職のリスクを予測する。

# サンプルデータ
data = {
    '年齢': [25, 45, 35, 50, 23, 40, 28, 50, 60, 36, 32, 25, 29, 41, 50, 38, 27, 46, 34, 49],
    '職務経験（年数）': [3, 20, 10, 30, 2, 15, 5, 29, 35, 11, 8, 3, 6, 18, 28, 12, 4, 21, 9, 25],
    '部署': ['営業', '技術', '人事', '技術', '営業', '人事', '営業', '技術', '営業', '技術', '人事', '営業', '営業', '技術', '人事', '技術', '営業', '技術', '人事', '技術'],
    '離職': [0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1]
}

dataset = pd.DataFrame(data)
print(dataset)

In [39]:
# 特徴量とラベルに分割
X = pd.get_dummies(dataset.iloc[:, :-1])
y = dataset.iloc[:, -1].values

# トレーニングデータとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# ランダムフォレストモデルのトレーニング
classifier = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=0)
classifier.fit(X_train, y_train)

# テストデータで予測
y_pred = classifier.predict(X_test)

# 混同行列の作成
cm = confusion_matrix(y_test, y_pred)
print(cm)

# 精度の計算
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy: .2f}')

**【課題】**

classifier = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=0)のコードが何を意味しているのかを調べて、説明せよ。

In [ ]:
#部分的な可視化を図る。

from sklearn.tree import export_graphviz
import graphviz

# ランダムフォレストの最初の決定木を可視化
estimator = classifier.estimators_[0]

# クラス名に対応する文字列のリストを作成
# classifier.classes_が返すのは [0, 1] なので、それぞれに対応する文字列を定義
class_labels = ['Not Left', 'Left']

dot_data = export_graphviz(
    estimator,
    out_file=None,
    feature_names=X.columns,
    class_names=class_labels,
    filled=True,
    rounded=True,
    special_characters=True
)
graph = graphviz.Source(dot_data)
graph.render("random_forest_tree")

# グラフを表示
graph

**【課題】**

他の決定木を用いてその結果をグラフ化し、それが何を示しているのかについて解釈を示せ。

# **【統合課題】**

今回学んだ3つ（4つ）のアルゴリズムの中で1つを選択し、他方で現実社会において公開されているデータを別途入手し、分類問題を解くコーディングを行え（colab上のpythonを用いて）。なおその際、可能な限り可視化を試み、より説得力のある形でプレゼンテーションを行うこと。

なお課題提出はipynbファイルで次回講義開始前までにslack上で行い、講義に際してはオンライン上で実行して結果を示すこと。